In [18]:
import qm
from entropylab import *
import numpy as np
import matplotlib.pyplot as plt

<div align="center"> <font size="+5"><u>Entropy: A lab workflow manager<\u></font></div>


Entropy is a tool for managing your quantum workflow. It allows you to build complex experiments, coordinating lab instruments and arbitrary computational steps. 
This notebook introduces the basic concepts needed to start working with Entropy.

# The Experiment and Graph

An experiment is a set of steps. 
These steps can depend on each other.
Each step is a node.

Each step needs must be traceable: we want to save data, we want to easily access data. 
A graph is a good representation.

# The Entropy Environment

An experiment has dependencies. 
The definition of a dependency is very broad: from an instrumnet, to software components, to calculations. 
These need to be shared between steps within the experiments and between different executions of experiments. 

To solve this we have the concept of a run environmet. 

It contains resources and is automatically visible to Nodes in an experiment graph.

# Results in Entropy

The results DB is an automatic and easy way to save and access results.

Entropy was designed to support different implementations of a DB. You can use a SQL DB, but don't have to. 
We will deploy several back ends, including for example to file based solutions such as HDF5

To support traceability, Entropy does not save only results, but metadata, debugging data etc. 

# Your first node

The PyNode is the basic element in an Entropy grpah. It runs arbitrary python code.
The function called by the node must return a dictionary with result names as keys. 



In [46]:
def my_func():
    return {'res':1}

node1 = PyNode("first_node", my_func,output_vars={'res'})
experiment = Graph(None, {node1}, "run_a") #No resources used here
handle = experiment.run()

2021-05-04 17:03:09,994 - entropy - WARNING - Results of current execution run_a will be permanently lost on session close
2021-05-04 17:03:09,995 - entropy - INFO - Running node <PyNode> first_node
2021-05-04 17:03:09,996 - entropy - INFO - Finished entropy experiment execution successfully



This node is now attached to a graph (containing just one node) and we run the graph, saving the results to `handle`

After running the experiment, you can access the results as following:

In [47]:
handle.results.get_results_from_node("first_node")[0].results[0].data

1

## Less simple

This example is too simple to show any real benefit. let's add resources.
We will now generate a `lab` which has a a database to which we save. 
This represents our entire lab, so in principle we only ever do this once and keep updating it as we go along. 



In [54]:

db = SqlAlchemyDB('docs_cache/tutorial.db')
lab = LabResources(db)


We now add two example resources. one represents a lab device - an oscilloscope and the other is a measurement from some thing

In [ ]:
asdkasld;

In [55]:
class my_thing:
    def __init__(self,name, val):
        self.name=name
        self.val=val

class my_scope:
    def __init__(self,name, ip):
        self.name=name
        self.ip=ip      
    
lab.register_resource("my_thing", my_thing,kwargs={'val':1,'name':'me'})
lab.register_resource("my_scope", my_scope,args=["gals scope"],experiment_kwargs={'ip':0})




KeyError: 'instrument my_thing already exist'

In [ ]:
def a(context: EntropyContext):
    x = 1
    y = 2
    print(context.get_resource("guy"))
    print(context.get_resource("tal"))
    print(x + y)
    return {"a_out": 10}


experiment_resources = ExperimentResources(db)
experiment_resources.import_lab_resource("guy")
experiment_resources.add_temp_resource("tal", 42)
# experiment_resources.add_temp_resource('gal',experiment_kwargs={'ip':123})
model = Script(experiment_resources, a, "running the exp")
script_experiment_handle = model.run(db)
exp_id = script_experiment_handle.id
results = script_experiment_handle.results
reader = ExperimentReader(exp_id, db)


In [149]:
reader.get_results()

[ResultRecord(experiment_id=1, id=1, label='experiment_result', story='Final output of the experiment', stage=-1, data={'a_out': 10})]

In [113]:
class my_int:
    def __init__(self,**kwargs):
        self.name=kwargs['name']
        self.val=kwargs['val']

In [117]:
my_int(**{'name':'me','val':1})

In [16]:
results